# Prueba: Ópalo Quant Capital

## 1 SQL

In [1]:
###################################################################################################
# Project    : Prueba Opalo Quant Capital
# Lenguaje   : Python / jupyter
# Description: Seccion de SQL
#
# References : 
# Git Control:
# Author - Year:Camilo Blanco Vargas - 2020
# Mail         :mail@camiloblanco.com 
###################################################################################################

In [2]:
# Import the necesary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sql

In [3]:
# Previo al ejercicio se crea una SQlite (archivo indicadores.db).
# Se crean las tablas de la DB y se alimenta con la data del archivo de excel ('prueba_opalo.xlsx').

# Connecting to indicadores empty DB on SQLite
from sqlalchemy import  create_engine
engine = create_engine("sqlite:///indicadores.db")
conn=engine.connect() # connect to the database
print(engine)

# Create a table on the DataBase
from sqlalchemy import Table, Column, Integer, Float, String, MetaData, String, ForeignKey,Text, DateTime, Boolean
from datetime import datetime
meta = MetaData()

# Declare indicadores table
indicadores = Table(
    'indicadores', meta, 
    Column('id_indicador', Integer, primary_key = True), 
    Column('nombre', String(50)), 
)

# Declare historico_indicadores table
historico_indicadores = Table(
    'historico_indicadores', meta, 
    Column('id_historico_indicador', Integer, primary_key = True), 
    Column('id_indicador', Integer, ForeignKey('indicadores.id_indicador')),
    Column('fecha', DateTime()),
    Column('valor', Float), 
)
# Create the tables on the DB
meta.create_all(engine)

# Read the db tables from the excel file to Pandas dataframes and clean them
myFile = 'prueba_opalo.xlsx'
tabla_indicadores = pd.read_excel(myFile, "tabla_indicadores")
tabla_historico_indicadores= pd.read_excel(myFile, "tabla_historico_indicadores")
tabla_historico_indicadores.drop(tabla_historico_indicadores.columns[0], axis=1,inplace = True) #Cleanning

# Insert records from the to Pandas dataframes to the DB
tabla_indicadores.to_sql('indicadores', conn, if_exists="append", index = False)
tabla_historico_indicadores.to_sql('historico_indicadores', conn, if_exists="append", index = False)

#Read from the SQL tables in Pandas datasframes and print
df_ind  = pd.read_sql('SELECT * FROM indicadores', con=engine)
print(df_ind .head(), "\n")
df_hist = pd.read_sql('SELECT * FROM historico_indicadores', con=engine)
print(df_hist.head())

Engine(sqlite:///indicadores.db)
   id_indicador           nombre
0             1      ipc_vigente
1             2  dtf_nta_vigente
2             3  ibr_nmv_vigente 

   id_historico_indicador  id_indicador                       fecha  valor
0                   16688             1  2020-12-01 00:00:00.000000   1.75
1                   16689             1  2020-12-02 00:00:00.000000   1.75
2                   16690             1  2020-12-03 00:00:00.000000   1.75
3                   16691             1  2020-12-04 00:00:00.000000   1.75
4                   16692             1  2020-12-05 00:00:00.000000   1.75


In [ ]:
# 1 sql
#### en un un pseudo codigo de python y  SQL: 
#### del excel adjunto importe los datos de la hoja "datos indicadores"
#### y actualice en postgres la "tabla_historico_indicador" lo anterior  teniendo en cuenta la "tabla_indicador"
#### ver foto adjunta en el mail
#### el nombre de la base de datos, la ruta y el usuario son cualquiera


In [4]:
# Importar los datos de la hoja "daros indicadores" y prepararlos para actulizar la tabla requerida
df_datos = pd.read_excel(myFile, "datos indicadores", skiprows=2)
df_datos=df_datos.melt(id_vars=['fecha'], value_vars=['dtf_nta_vigente', 'ibr_nmv_vigente','ipc_vigente'])
df_datos.columns = ['fecha','nombre','valor']
#teniendo en cuenta la "tabla_indicador"...(df_ind)
df_datos=pd.merge(df_datos,df_ind, on='nombre') #Join...
df_datos.drop(df_datos.columns[1], axis=1,inplace = True) #Cleanning
print(df_datos.head())
print(df_datos.tail())

       fecha  valor  id_indicador
0 2020-12-06   1.94             2
1 2020-12-07   1.93             2
2 2020-12-08   1.93             2
3 2020-12-09   1.93             2
4 2020-12-10   1.93             2
        fecha  valor  id_indicador
25 2020-12-11   1.49             1
26 2020-12-12   1.49             1
27 2020-12-13   1.49             1
28 2020-12-14   1.49             1
29 2020-12-15   1.49             1


In [5]:
# Actualice la "tabla_historico_indicadores"
for index, row in df_datos.iterrows():  
    id_ind=int(row['id_indicador'])
    date=str(row['fecha'])+'.000000'
    val=float(row['valor'])
    conn.execute('UPDATE historico_indicadores SET valor = :x WHERE id_indicador=:y AND fecha=:z',
             x=val, y = id_ind, z=date)
    
# Query desde la tabla actualizada...
df_hist = pd.read_sql('SELECT * FROM historico_indicadores WHERE fecha BETWEEN "2020-12-06" AND "2020-12-16"', con=engine)
print(df_hist.head(30))

    id_historico_indicador  id_indicador                       fecha  valor
0                    16693             1  2020-12-06 00:00:00.000000  1.490
1                    16694             1  2020-12-07 00:00:00.000000  1.490
2                    16695             1  2020-12-08 00:00:00.000000  1.490
3                    16696             1  2020-12-09 00:00:00.000000  1.490
4                    16697             1  2020-12-10 00:00:00.000000  1.490
5                    16698             1  2020-12-11 00:00:00.000000  1.490
6                    16699             1  2020-12-12 00:00:00.000000  1.490
7                    16700             1  2020-12-13 00:00:00.000000  1.490
8                    16701             1  2020-12-14 00:00:00.000000  1.490
9                    16702             1  2020-12-15 00:00:00.000000  1.490
10                   17023             3  2020-12-06 00:00:00.000000  1.717
11                   17024             3  2020-12-07 00:00:00.000000  1.718
12          

# 2 sql
#### en un un pseudo codigo de python y  SQL: 
####  inserte en postgres en la "tabla_historico_indicador" los siguientes valores 

#### fecha = 01-01-2021
#### ipc_vigente = 1.49
#### ibr_nmv_vigente = 1.714
#### dtf_nta_vigente = 11.92
#### el nombre de la base de datos, la ruta y el usuario son cualquiera

In [6]:
# Insertar los nuevos datos en la base de datos:
conn.execute('INSERT INTO historico_indicadores(id_indicador,fecha,valor)  VALUES(1,"01-01-2021 00:00:00.000000",1.49)')
conn.execute('INSERT INTO historico_indicadores(id_indicador,fecha,valor)  VALUES(2,"01-01-2021 00:00:00.000000",11.92)')
conn.execute('INSERT INTO historico_indicadores(id_indicador,fecha,valor)  VALUES(3,"01-01-2021 00:00:00.000000",1.714)')
# Query desde la tabla actualizada...
df_hist = pd.read_sql('SELECT * FROM historico_indicadores WHERE fecha = "01-01-2021 00:00:00.000000"', con=engine)
print(df_hist.head())

   id_historico_indicador  id_indicador                       fecha   valor
0                   17405             1  01-01-2021 00:00:00.000000   1.490
1                   17406             2  01-01-2021 00:00:00.000000  11.920
2                   17407             3  01-01-2021 00:00:00.000000   1.714


# 3 sql
#### en un un pseudo codigo de python y  SQL: 
####  elimine  en postgres en  la "tabla_historico_indicador" los datos del 30 de de noviembre del 2020
#### el nombre de la base de datos, la ruta y el usuario son cualquiera

In [ ]:
# En la tabla suministrada los datos inician en diciembre de 2020... sin embargo el query seria:
conn.execute('DELETE FROM historico_indicadores WHERE fecha = "30-11-2020 00:00:00.000000"')